In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('../../../module/')

from keras2.models import Model
from keras2.layers import concatenate, Dense, Input, Flatten
from keras2.optimizers import Adam, Optimizer
import keras2.backend as K
import tensorflow as tf
import csv
import itertools
from util import *
import gym2
from rl2.agents import selfDDPGAgent, selfDDPGAgent2, selfDDPGAgent3
from rl2.memory import SequentialMemory

Using TensorFlow backend.
Using TensorFlow backend.


In [2]:
env = gym2.make('Linear-v1')
Q = .01 * np.eye(2)
R = .01

In [86]:
def critic_net(a_shape , s_shape):
    action_input = Input(a_shape)
    observation_input = Input(shape=(1,)+s_shape)
    flattened_observation = Flatten()(observation_input)
    x = concatenate([action_input, flattened_observation])
    x = Dense(16, activation="relu")(x)
    x = Dense(16, activation="relu")(x)
    x = Dense(1, activation="linear")(x)
    critic = Model(inputs=[action_input, observation_input], outputs=x)
    return (critic, action_input)

def value_net(s_shape):
    state_input = Input((1,)+s_shape)
    x = Flatten()(state_input)
    
    x = Dense(16, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    x = Dense(1, activation="linear")(x)
    value = Model(inputs=state_input, output=x)
    return value

def branch_actor(a_shape, s_shape):
    action_input = Input(shape=(1,)+s_shape)
    x = Flatten()(action_input) # 実質的なinput layer
    
    x1 = Dense(16, activation="relu")(x)
    x1 = Dense(16, activation="relu")(x1)
    x1 = Dense(1, activation="multiple_tanh")(x1) # action signal
    
    x2 = Dense(16, activation="relu")(x)
    x2 = Dense(16, activation="relu")(x2)
    x2 = Dense(1, activation="tau_output_large")(x2) # tau
    
    output = concatenate([x1, x2])
    actor = Model(inputs=action_input, outputs=output)
    return actor

In [87]:
actor = branch_actor((2,),(2,))
critic = critic_net((2,),(2,))[0]
critic.compile(Adam(lr=0.001, clipnorm=1.), loss='mean_squared_error')

In [88]:
x = tf.placeholder(tf.float32, shape=(None, 1))
y = tf.placeholder(tf.float32, shape=(None, 1))
d = tf.matmul(x,y, transpose_a=True)

print(x)
print(d)

with tf.Session() as sess:
    print(sess.run(d, feed_dict={x:[[0.3],[0.2]], y:[[3],[4]]}))

Tensor("Placeholder_91:0", shape=(?, 1), dtype=float32)
Tensor("MatMul_16:0", shape=(1, 1), dtype=float32)
[[1.7]]


In [89]:
print(np.array([[[1]],[[2]]]).shape)

(2, 1, 1)


In [118]:
alpha = 0.4
beta = 1.
dummy_optimizer = Optimizer()
def gradient_for_one_data(actor, critic):
    params = actor.trainable_weights  
    # q_func gradient tensor
    state_input = tf.placeholder(tf.float32, shape=(1, 1, 2))
    actor_output = actor(state_input)
    combined_input_tensor = [actor_output, state_input]
    critic_output_tensor = critic(combined_input_tensor)
    loss = -K.mean(critic_output_tensor)
    gradient_for_q = dummy_optimizer.get_gradients(loss, params)
    
    # discount factor gradient tensor
    discount = - tf.exp(- alpha * actor_output[0])
    gradient_for_d = dummy_optimizer.get_gradients(discount, params)
    next_state_input = tf.placeholder(tf.float32, shape=(1, 1, 2))
    next_action = actor(next_state_input)
    next_value = critic([next_action, next_state_input])[0]
    gradient_for_d = [next_value * g for g in gradient_for_d]
    
    gradient_for_one_state = [K.add(gq, gd) for gq, gd in zip(gradient_for_q, gradient_for_d)]
    
    return gradient_for_one_state

def gradient(states, ts, next_states, actor, critic):
    batch_size = len(states)
    discounts = np.exp(- alpha * np.array(ts))
    g_tensor = gradient_for_one_data(actor, critic)
    out = [tf.constant(0.)] * batch_size
    
    for i in range(batch_size):
        # 割引率 * 勾配 を 足していく
        out = [K.add(o, discounts[i] * g) for o, g in zip(out, g_tensor)]
        
    
    return out

In [103]:
state = np.array([1,2])
next_state = np.array([2,3])
g = gradient_for_one_data(actor, critic)

In [121]:
states = [np.array([1,2]), np.array([1,2])]
ts = np.random.uniform(2)
gradient(states,ts,_,actor,critic)

IndexError: invalid index to scalar variable.

In [ ]:
np.e